In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# import file

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_train=pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
df_test=pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
submit=pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
submit.head()

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
#check missing value
print(df_train.isnull().sum())
print(df_test.isnull().sum())

In [ ]:
#check data type
df_train.info()
df_test.info()

In [ ]:
#summary statistic
df_train.describe()

In [ ]:
df_test.describe()

# histgram

In [ ]:
#Target-histgram
sns.set()
plt.hist(df_train["pressure"])
plt.xlabel("pressure")
plt.ylabel("Frequency")
plt.show()

In [ ]:
#Feature histgram
for i in df_test.columns:
    plt.hist(df_train[i], alpha=0.5, label="train")
    plt.hist(df_test[i], alpha=0.5, label="test")
    plt.xlabel(i)
    plt.ylabel("Frequency")
    plt.legend(loc=1)
    plt.show()

# relationship for pressure (boxplot & scatter plot)

In [ ]:
x="R"
sns.boxplot(df_train[x], df_train["pressure"])
plt.xlabel(x)
plt.ylabel("pressure")
plt.show()

In [ ]:
x="C"
sns.boxplot(df_train[x], df_train["pressure"])
plt.xlabel(x)
plt.ylabel("pressure")
plt.show()

In [ ]:
x="time_step"
plt.scatter(df_train[x], df_train["pressure"])
plt.xlabel(x)
plt.ylabel("pressure")
plt.show()

In [ ]:
x="u_in"
plt.scatter(df_train[x], df_train["pressure"])
plt.xlabel(x)
plt.ylabel("pressure")
plt.show()

In [ ]:
x="u_out"
sns.boxplot(df_train[x], df_train["pressure"])
plt.xlabel(x)
plt.ylabel("pressure")
plt.show()

# breath ID

In [ ]:
#number of unique value
print(df_train["breath_id"].nunique())
print(df_test["breath_id"].nunique())

In [ ]:
df_train["breath_id"]

In [ ]:
def lineplot(df):
    plt.plot(df["time_step"], df["pressure"])
    plt.xlabel("time_step")
    plt.ylabel("pressure")
    plt.show()

In [ ]:
breath1=df_train.loc[df_train["breath_id"]==1]
breath2=df_train.loc[df_train["breath_id"]==2]
breath3=df_train.loc[df_train["breath_id"]==3]
breath4=df_train.loc[df_train["breath_id"]==4]

In [ ]:
lineplot(breath1)

In [ ]:
lineplot(breath2)

In [ ]:
lineplot(breath3)

In [ ]:
lineplot(breath4)

In [ ]:

plt.plot(breath1["time_step"], breath1["pressure"])
plt.xlabel("time_step")
plt.ylabel("pressure")
plt.show()

# Simple Model

In [ ]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, StratifiedKFold, GroupKFold, KFold, train_test_split

from tqdm import tqdm_notebook as tqdm

In [ ]:
df_train.head(2)

In [ ]:
y_train=df_train["pressure"]
x_train=df_train.drop(["id", "breath_id", "pressure"], axis=1)


In [ ]:
x_test=df_test.drop(["id","breath_id"], axis=1)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
#GroupID for Group-KFold
groups = df_train["breath_id"]
groups

In [ ]:
# CV Averaging
scores = []
y_pred_test = np.zeros(len(x_test)) #array for predict value
gkf = GroupKFold(n_splits=5)

for i, (train_ix, test_ix) in tqdm(enumerate(gkf.split(x_train, y_train, groups))):
    
    X_train_, y_train_, groups_train_ = x_train.iloc[train_ix], y_train.iloc[train_ix], groups[train_ix]
    X_val, y_val, groups_val = x_train.iloc[test_ix], y_train.iloc[test_ix], groups[test_ix]
    
    print('Train Groups', np.unique(groups_train_))
    print('Val Groups', np.unique(groups_val))
    print(X_train_.shape, X_val.shape)
    
    model = lgb.LGBMRegressor(random_state=71, importance_type='gain')
    
    model.fit(X_train_, y_train_)
    y_pred = model.predict(X_val)
    
    y_pred_test += model.predict(x_test) # add predict value
    
    score =  mean_absolute_error(y_val, y_pred)
    scores.append(score) 
    
    
    print('CV Score of Fold_%d is %f' % (i, score))
    
   
   

In [ ]:
print(scores)
print(np.mean(scores))

In [ ]:
y_pred_test_submit=y_pred_test/5 #n_splits=5


In [ ]:
y_pred_test_submit

In [ ]:
submit["pressure"]=y_pred_test_submit
submit.head()

In [ ]:
submit.to_csv("submission.csv",index=False)

# Feature Impact

In [ ]:
importance = pd.DataFrame(model.feature_importances_, index=x_train.columns, columns=['importance'])
importance

In [ ]:
sns.barplot(x=importance.index, y="importance", data=importance)

In [ ]:
#End